In [67]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import requests
import json
import random

In [97]:
df = pd.read_json("data.json", lines=True)
# df = df.sample(3000)
df.shape

(209527, 6)

In [98]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [99]:
df["category"].value_counts() # i'll do the top 10

POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9900
STYLE & BEAUTY     9814
PARENTING          8791
HEALTHY LIVING     6694
QUEER VOICES       6347
FOOD & DRINK       6340
BUSINESS           5992
COMEDY             5400
SPORTS             5077
BLACK VOICES       4583
HOME & LIVING      4320
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3653
WOMEN              3572
CRIME              3562
IMPACT             3484
DIVORCE            3426
WORLD NEWS         3299
MEDIA              2944
WEIRD NEWS         2777
GREEN              2622
WORLDPOST          2579
RELIGION           2577
STYLE              2254
SCIENCE            2206
TECH               2104
TASTE              2096
MONEY              1756
ARTS               1509
ENVIRONMENT        1444
FIFTY              1401
GOOD NEWS          1398
U.S. NEWS          1377
ARTS & CULTURE     1339
COLLEGE            1144
LATINO VOICES      1130
CULTURE & ARTS     1074
EDUCATION       

In [100]:
df[["headline", "category"]]

,headline,category
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY
3,The Funniest Tweets From Parents This Week (Se...,PARENTING
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS
...,...,...
209522,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH
209523,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS
209524,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS
209525,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS


In [101]:
df['category']=df['category'].replace({"HEALTHY LIVING": "WELLNESS",
"QUEER VOICES": "GROUPS VOICES",
"BUSINESS": "BUSINESS & FINANCES",
"PARENTS": "PARENTING",
"BLACK VOICES": "GROUPS VOICES",
"THE WORLDPOST": "WORLD NEWS",
"STYLE": "STYLE & BEAUTY",
"GREEN": "ENVIRONMENT",
"TASTE": "FOOD & DRINK",
"WORLDPOST": "WORLD NEWS",
"SCIENCE": "SCIENCE & TECH",
"TECH": "SCIENCE & TECH",
"MONEY": "BUSINESS & FINANCES",
"ARTS": "ARTS & CULTURE",
"COLLEGE": "EDUCATION",
"LATINO VOICES": "GROUPS VOICES",
"CULTURE & ARTS": "ARTS & CULTURE",
"FIFTY": "MISCELLANEOUS",
"GOOD NEWS": "MISCELLANEOUS"})

In [102]:
df.drop_duplicates(['headline','short_description'], keep = 'last', inplace = True)

In [103]:
balanced_data = pd.DataFrame() # getting the top 10 and balancing
#5077 sports
from sklearn.utils import resample
for i in ["POLITICS", "WELLNESS", "ENTERTAINMENT",  "PARENTING", "STYLE & BEAUTY", "GROUPS VOICES", "TRAVEL", "WORLD NEWS", "FOOD & DRINK", "BUSINESS & FINANCES", "COMEDY", "SPORTS"]:
    row = df[df["category"] == i]
    row = (resample(row, replace=False, n_samples=5000, random_state=123))
    balanced_data = pd.concat([balanced_data, row])
    
balanced_data["category"].value_counts() # ok now it's balanced
df = balanced_data

In [104]:
df["texts"] = df["headline"] + " " + df["short_description"]
df.drop(["headline", "short_description"], inplace=True, axis=1)
texts = df["texts"]
labels = df["category"]

In [105]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

model = make_pipeline(TfidfVectorizer(), MultinomialNB())

In [106]:
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.4, random_state=42)
model.fit(texts_train, labels_train)
pred_labels = model.predict(texts_test)

In [107]:
accuracy = accuracy_score(pred_labels, labels_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.712625


In [111]:

print(model.predict(["MLS Cup Highlights: Columbus Crew SC vs. LAFC | December 9, 2023 | 📺 Watch every match with MLS Season Pass on Apple TV: http://apple.co/MLSThe 🐐 plays here. Join The Messi Insider for insider news, early access to Messi p...."]))

['SPORTS']


In [110]:
import pickle

pickle.dump(model, open('model.pkl', 'wb'))